In [1]:
#ALWAYS run the first 2 blocks of code. ALWAYS!
import pandas as pd
import numpy as np
import numpy.linalg as linalg
import numpy.random as rnd
import matplotlib.pyplot as plt
import zipfile
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas_datareader as pdr
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import time
import re

In [2]:
%cd downloads

/Users/varunginde/Downloads


In [3]:
driver = webdriver.Chrome('/usr/local/bin/chromedriver version 91')

## Scraping Financial Statements
For the purpose of valuation, I'll mainly need the balance sheet, income statement and the various ratios of the company. The browser controlled by Selenium will scrape this data from _moneycontrol_. This scraper is meant for companies which have been listed for at least 5 years.

In [4]:
def stock_ki_kundali(stock_name,params,con=True):
    df_list = []

    if type(params)==str:
        params = [params]
    for param in params:
        driver.get('https://www.google.com/')
        soup = BeautifulSoup(driver.page_source,'lxml')
        
        search_box = driver.find_element_by_name('q') 
        search_box.send_keys(f"moneycontrol {stock_name} {param}") 
        search_box.submit()
    
        soup_1 = BeautifulSoup(driver.page_source,'lxml')
    
        s = soup_1.find_all('div',{'class':'yuRUbf'})[0]
        soup_2 = s.find_all('a',href=True)
        driver.get((soup_2[0]['href']))
        
        soup = BeautifulSoup(driver.page_source,'lxml')
        if con==True:
            for i in soup.find_all('a'):
                if 'Consolidated' in i.text:
                    driver.get(i['href'])
                    break
        #else:
            #for i in soup.find_all('a'):
                #if 'Standalone' in i.text:
                    #driver.get(i['href'])
                    #break
    
        soup = BeautifulSoup(driver.page_source,'lxml')
        titles = []
        numbers = []
        l = []
        if param not in ['ratios','profit and loss statement']:
            for data in soup.find_all('tr',{'class':'lightbg'}):
                for nos in data.find_all('td'):
                    l.append(nos.text)
        else:
            for data in soup.find_all('tr'):
                for nos in data.find_all('td'):
                    l.append(nos.text)
            l = l[1:]
            
        for i in range(len(l)//7):
            titles.append(l[7*i])
            numbers.append(l[7*i+1:7*i+6])
        df = pd.DataFrame(numbers,index=titles)
        header = df.iloc[0]
        df.drop(df.index[0],axis=0,inplace=True)
        df.columns = header
        for col in df.columns:
            try:
                df[col] = df[col].apply(lambda x:float(x.replace(',','')))
            except ValueError:
                pass
        df_list.append(df)
        
    return df_list

In [5]:
kundali = stock_ki_kundali('Infosys',['ratios','profit and loss statement','balance sheet'])
df1 = kundali[0]
df2 = kundali[1]
df3 = kundali[2]

The 3 dataframes above consist of the data for Infosys. The 1st one consists of the various ratios, the 2nd consists of the income statement and the 3rd consists of the balance sheet.

In [6]:
df2

Profit & Loss account of Infosys (in Rs. Cr.),Mar 21,Mar 20,Mar 19,Mar 18,Mar 17
,12 mths,12 mths,12 mths,12 mths,12 mths
INCOME,,,,,
Revenue From Operations [Gross],"100,472.00","90,791.00","82,675.00","70,522.00","68,484.00"
Less: Excise/Sevice Tax/Other Levies,0.00,0.00,0.00,0.00,0.00
Revenue From Operations [Net],"100,472.00","90,791.00","82,675.00","70,522.00","68,484.00"
Total Operating Revenues,"100,472.00","90,791.00","82,675.00","70,522.00","68,484.00"
Other Income,"2,201.00","2,803.00","2,882.00","3,193.00","3,080.00"
Total Revenue,"102,673.00","93,594.00","85,557.00","73,715.00","71,564.00"
EXPENSES,,,,,
Cost Of Materials Consumed,0.00,0.00,0.00,0.00,0.00


## The Capital Asset Pricing Model (CAPM)
I'm going to value Infosys by discounting its cash flows. In order to calculate the discount rate,  I'll use the CAPM. As per the CAPM, the expected rate of return for an asset can be written as :

$$r = R_f + \beta(Equity Risk Premium)$$

Here, the R_f denotes the risk-free rate of interest. For the US (and any other market which is mature), I have assumed the risk-free rate to be the 10-year T Bill rate. The $\beta$ is the measure of the volatility or the risk. By definition, it is the ratio of the covariance of the company's returns and the market returns (Nifty50 in this case), to the variance of the returns of the market. The equity risk premium is estimated by subtracting the expected return on bonds from the expected return of the equity market. This sheet (updated Jan 2021) consists of ERP data for most countries. 

In [7]:
erpdata2021 = pd.read_excel('ctryprem.xlsx',sheet_name=6)
erpdata2021.set_index('Country',inplace=True)

In [8]:
erpdata2021.loc['India',:]

GDP (in billions) in 2019      2875.1
Moody's rating                   Baa3
Sovereign CDS                  0.0124
Adj. Default Spread          0.019453
Equity Risk Premium          0.068519
Country Risk Premium         0.021319
Corporate Tax Rate                0.3
Region                           Asia
Name: India, dtype: object

The 2 dataframes below consist of the data for Indian companies and the $\beta$ value for each sector. 

In [9]:
companies2021 = pd.read_excel('indname.xls',sheet_name=3)
companies2021[companies2021['Company Name'].str.match('(^Deepak Nitrite.*)')==True]

,Company Name,Exchange:Ticker,Industry Group,Country,Broad Group,Sub Group
3684,Deepak Nitrite Limited (BSE:506401),BSE:506401,Chemical (Basic),India,Emerging Markets,India


In [10]:
betas2021 = pd.read_excel('totalbetaIndia.xls',skiprows=6)
betas2021.columns = betas2021.iloc[0,:]
betas2021 = betas2021.drop(0)
betas2021.set_index('Industry Name',inplace=True)

In [11]:
betas2021.loc['Chemical (Basic)',:]

0
Number of firms                             130
Average Unlevered Beta                  1.03998
Average Levered Beta                    1.11706
Average correlation with the market    0.247883
Total Unlevered Beta                    4.19544
Total Levered Beta                      4.50639
Name: Chemical (Basic), dtype: object

The class below does most of the heavy lifting. It takes in the company data as well as the 3 dataframes above in order to estimate various important inputs that will be needed for valuation. Depending on the company, I'll decide whether to use dividends, free cash flows to equity or free cash flows to the firm. 

In [12]:
class valuer:
    def __init__(self,beta_df,comp_df,erp_df,comp,ratios,pnl,balance,capex,shares,risk_free_rate=0.0429,dcf_type='firm'):
        self.beta_df = beta_df
        self.comp_df = comp_df
        self.erp_df = erp_df
        self.comp = comp
        self.ratios = ratios
        self.pnl = pnl
        self.balance = balance
        self.capex = capex
        self.risk_free_rate = risk_free_rate
        self.dcf_type = dcf_type
        self.shares = shares
        
    def t(self):
        return self.erp_df.loc['India','Corporate Tax Rate']
    
    def change_wc(self):
        wc_pres=self.balance.loc['Total Current Assets','Mar 21'] - self.balance.loc['Total Current Liabilities','Mar 21']
        wc_past=self.balance.loc['Total Current Assets','Mar 20'] - self.balance.loc['Total Current Liabilities','Mar 20']
        return (wc_pres - wc_past)
    
    def DE(self):
        return float(self.ratios.loc['Total Debt/Equity (X)','Mar 21'])
    
    def DR(self):
        de = float(self.ratios.loc['Total Debt/Equity (X)','Mar 21'])
        return de/(1+de)
    
    def dep(self):
        return float(self.pnl.loc['Depreciation And Amortisation Expenses','Mar 21'].replace(',',''))
    
    def net_capex(self):
        return self.capex - float(self.pnl.loc['Depreciation And Amortisation Expenses','Mar 21'].replace(',',''))
    
    def EBIT(self):
        try:
            pbt = float(self.pnl.loc['Profit/Loss Before Tax','Mar 21'].replace(',',''))
            ie = float(self.pnl.loc['Finance Costs','Mar 21'].replace(',',''))
            return pbt+ie
        except KeyError:
            pbt = float(self.pnl.loc['Net Profit / Loss for The Year'].replace(',',''))
            ie = float(self.pnl.loc['Finance Costs','Mar 21'].replace(',',''))
            return pbt+ie
    
    def FCFF(self):
        return self.EBIT()*(1-self.t()) - self.net_capex() - self.dep()
    
    def reinvestment(self):
        if self.dcf_type=='firm':
            return (self.net_capex() + self.dep())/(self.EBIT()*(1-self.t()))
        elif self.dcf_type=='equity':
            ni = float(self.pnl.loc['Profit/Loss For The Period','Mar 21'].replace(',',''))
            return (1-self.DR())*(self.net_capex()+self.change_wc())/ni
    
    def rev_growth(self):
        rev_recent = float(self.pnl.loc['Depreciation And Amortisation Expenses','Mar 21'].replace(',',''))
        rev_past = float(self.pnl.loc['Depreciation And Amortisation Expenses','Mar 17'].replace(',',''))
        return (rev_recent/rev_past)**0.25 - 1
    
    def FCFE(self):
        ni = float(self.pnl.loc['Profit/Loss For The Period','Mar 21'].replace(',',''))
        return ni - (1-self.DR())*(self.net_capex()+self.change_wc())
    
    def crp(self):
        return self.erp_df.loc['India','Country Risk Premium'] 
    
    def erp(self):
        return self.erp_df.loc['India','Equity Risk Premium'] 
    
    def mature_market_prem(self):
        return self.erp_df.loc['United States','Equity Risk Premium']
    
    def sector(self):
        s1 = self.comp_df[self.comp_df['Company Name'].str.match(f'(^{self.comp}.*)')==True]
        a = s1['Industry Group']
        return a.iloc[0]
    
    def industry_beta_unlevered(self):
        b = self.beta_df.loc[self.sector(),'Average Unlevered Beta']
        return b
        
    def beta(self):
        b = self.industry_beta_unlevered()*(1+(1-self.t())*self.DE())
        return b
    
    def r_equity(self,model='capm',l=1):
        if model=='capm':
            return self.risk_free_rate + self.beta()*self.erp()
        elif model == 'modified capm':
            return self.risk_free_rate + self.crp() + self.beta()*self.mature_market_prem()
        elif model == 'lambda':
            return self.risk_free_rate + l*self.crp() + self.beta()*self.mature_market_prem()
    
    def r_debt(self):
        spread = self.erp_df.loc['India','Adj. Default Spread']
        return (self.risk_free_rate + spread)*(1-self.t())    
    
    def r(self):
        if self.dcf_type=='firm':
            d = self.DR()
            e = 1 - d
            return e*self.r_equity() + d*self.r_debt()
        elif self.dcf_type in ['equity','dividend']:
            return self.r_equity()
        
    def roe(self):
        return (self.ratios.loc['Return on Networth/Equity (%)',:].astype(float)).mean()/100
    
    def roc(self):
        return (self.ratios.loc['Return on Capital Employed (%)',:].astype(float)).mean()/100
    
    def avg_sector_DE(self):
        b_unlevered = self.beta_df.loc[self.sector(),'Average Unlevered Beta']
        b_levered = self.beta_df.loc[self.sector(),'Average Levered Beta']
        return ((b_levered/b_unlevered) - 1)/(1-self.t())
    
    def r_stable(self,model='capm',l=1):
        b = self.beta_df.loc[self.sector(),'Average Levered Beta']
        m = model
        if m == 'capm':
            r_eq = self.risk_free_rate + b*self.erp()
        elif m == 'modified capm':
            r_eq = self.risk_free_rate + self.crp() + b*self.mature_market_prem()
        elif m == 'lambda':
            r_eq = self.risk_free_rate + l*self.crp() + b*self.mature_market_prem()
            
        d = self.avg_sector_DE()/(1+self.avg_sector_DE())
        e = 1 - d
        if self.dcf_type=='firm':
            return d*self.r_debt() + e*r_eq
        elif self.dcf_type in ['equity','dividend']:
            return r_eq
        
    def stock_value(self,time=5):
        if self.dcf_type=='firm':
            g = self.reinvestment()*self.roc()
            g_stable = round(100*self.risk_free_rate)/100
            s = 0
            for i in range(1,1+time):
                s += self.FCFF()*((1+g)/(1+self.r()))**i
            terminal_val = (1+g_stable)*self.FCFF()*((1+g)/(1+self.r()))**5
            terminal_val = terminal_val/(self.r_stable() - g_stable)
            return (terminal_val + s)*(10**7)/self.shares
    
    def growth_ni(self):
        ni_series = self.pnl.loc['Profit/Loss For The Period',:].apply(lambda s : float(s.replace(',','')))
        return (ni_series[0]/ni_series[-1])**0.25 - 1

## Infosys

In [38]:
infy_data = valuer(betas2021,companies2021,erpdata2021,'Infosys',df1,df2,df3,2107,dcf_type='firm',shares=551682338*100/12.95)

In [39]:
infy_data.pnl

Profit & Loss account of Infosys (in Rs. Cr.),Mar 21,Mar 20,Mar 19,Mar 18,Mar 17
,12 mths,12 mths,12 mths,12 mths,12 mths
INCOME,,,,,
Revenue From Operations [Gross],"100,472.00","90,791.00","82,675.00","70,522.00","68,484.00"
Less: Excise/Sevice Tax/Other Levies,0.00,0.00,0.00,0.00,0.00
Revenue From Operations [Net],"100,472.00","90,791.00","82,675.00","70,522.00","68,484.00"
Total Operating Revenues,"100,472.00","90,791.00","82,675.00","70,522.00","68,484.00"
Other Income,"2,201.00","2,803.00","2,882.00","3,193.00","3,080.00"
Total Revenue,"102,673.00","93,594.00","85,557.00","73,715.00","71,564.00"
EXPENSES,,,,,
Cost Of Materials Consumed,0.00,0.00,0.00,0.00,0.00


In [40]:
infy_data.r_stable()

0.09618015125442662

In [41]:
infy_data.stock_value()

708.247808351289

In [42]:
infy_data.EBIT()

26823.0

In [43]:

infy_data.FCFF()

16669.1

In [44]:
infy_data.r()

0.0959240298722952

In [45]:
infy_data.avg_sector_DE()

-0.01661372319385901

In [46]:
infy_data.r_debt()

0.043647076013418065

In [47]:
infy_data.DE()

0.0

In [48]:
infy_data.FCFE()

17435.0

In [49]:
infy_data.rev_growth()

0.17688389992172038

In [50]:
ni_series = infy_data.pnl.loc['Profit/Loss For The Period',:].apply(lambda s : float(s.replace(',','')))

In [51]:
growth_ni = (ni_series[0]/ni_series[-1])**0.25 - 1

In [52]:
growth_ni

0.07799490303428502

In [53]:
infy_data.risk_free_rate

0.0429

In [54]:
infy_data.erp()

0.06851895817076799

In [55]:
infy_data.beta()

0.7738592542540534

In [56]:
#Assuming stable growth
shares = 560182338*100/12.84
value = infy_data.FCFE()*(1+growth_ni)/(infy_data.r_equity() - growth_ni)

In [57]:
r1 = (1+infy_data.growth_ni())/(1+infy_data.r_equity())

In [58]:
infy_data.change_wc()

3148.0

In [59]:
infy_data.net_capex()

-1160.0

In [60]:
infy_data.dep()

3267.0

In [61]:
infy_data.shares

4260095274.131274

In [64]:
tot = infy_data.FCFE()*r1*(1 - r1**5)/(1-r1) + infy_data.FCFE()*(r1**5)*(1.05)/(infy_data.r_equity() - 0.05)

In [65]:
per_share_value = (10**7)*value/infy_data.shares
print(f'{infy_data.comp} has a price per share of ₹{round(per_share_value,2)}')

Infosys has a price per share of ₹2460.71


## HCL

In [101]:
kundali_tcs = stock_ki_kundali('HCL Technologies',['ratios','profit and loss statement','balance sheet'])
hcl_df1 = kundali_tcs[0]
hcl_df2 = kundali_tcs[1]
hcl_df3 = kundali_tcs[2]

In [104]:
hcl_df3

Balance Sheet of HCL Technologies (in Rs. Cr.),Mar 21,Mar 20,Mar 19,Mar 18,Mar 17
Total Share Capital,543.0,543.0,271.0,278.0,285.0
Total Reserves and Surplus,59370.0,50724.0,41095.0,36108.0,32664.0
Total Shareholders Funds,59913.0,51267.0,41366.0,36386.0,32949.0
Total Non-Current Liabilities,8729.0,7755.0,4807.0,1530.0,1305.0
Total Current Liabilities,17383.0,23730.0,12299.0,10107.0,11335.0
Total Capital And Liabilities,86194.0,82906.0,58575.0,48023.0,45762.0
Fixed Assets,20265.0,21736.0,14062.0,12274.0,9179.0
Total Non-Current Assets,43143.0,44486.0,28853.0,23465.0,19299.0
Total Current Assets,43051.0,38420.0,29722.0,24558.0,26463.0
Total Assets,86194.0,82906.0,58575.0,48023.0,45762.0


In [105]:
hcl_data = valuer(betas2021,companies2021,erpdata2021,'HCL Technologies',hcl_df1,hcl_df2,hcl_df3,1964.6,dcf_type='firm',shares=2713665096)

In [106]:
hcl_data.r()

0.09506590913937424

In [107]:
hcl_data.FCFE()

3309.0

In [108]:
hcl_data.FCFF()

9490.199999999999

In [109]:
ni_series_hcl = hcl_data.pnl.loc['Profit/Loss For The Period',:].apply(lambda s : float(s.replace(',','')))

In [110]:
ni_series_hcl

Profit & Loss account of HCL Technologies (in Rs. Cr.)
Mar 21    11169.0
Mar 20    11057.0
Mar 19    10120.0
Mar 18     8709.0
Mar 17     8604.0
Name: Profit/Loss For The Period, dtype: float64

In [111]:
hcl_data.DE()

0.06

In [113]:
hcl_data.FCFF()

9490.199999999999

In [114]:
hcl_data.rev_growth()

0.5361767803172597

In [115]:
(ni_series_hcl[0]/ni_series_hcl[-1])**0.25 - 1

0.06740313081751359

In [125]:
g = hcl_data.roc()*hcl_data.reinvestment()

In [123]:
hcl_data.reinvestment()

0.1715088871040961

In [119]:
hcl_data.FCFE()

3309.0

In [124]:
hcl_data.roc()

0.27198

In [120]:
hcl_data.r()

0.09506590913937424

In [121]:
hcl_data.r_equity()

0.0981510391269316

In [126]:
hcl_data.FCFF()

9490.199999999999

In [127]:
hcl_data.EBIT()

16364.0

In [129]:
rat = (1+g)/(1+hcl_data.r())

In [130]:
hcl_data.shares

2713665096

In [133]:
total_val = hcl_data.FCFF()*rat*(1 - rat**5)/(1-rat) + hcl_data.FCFF()*(1.045)**6/(hcl_data.r() - 0.045)

In [135]:
total_val*(10**7)/hcl_data.shares

1062.6387013473607

## Exide Industries

In [16]:
ekundali = stock_ki_kundali('Exide Industries Limited',['ratios','profit and loss statement','balance sheet'])
ex_df1 = ekundali[0]
ex_df2 = ekundali[1]
ex_df3 = ekundali[2]

#infy_data = valuer(betas2021,companies2021,erpdata2021,'Infosys',df1,df2,df3,2107,dcf_type='firm',shares=560182338)

In [30]:
ex_data = valuer(betas2021,companies2021,erpdata2021,'Exide Industries Limited',ex_df1,ex_df2,ex_df3,capex=501.62,dcf_type='firm',shares=850000000)

In [31]:
ex_data.r()

0.1369196335510024

In [32]:
ex_data.r_equity()

0.13878508470175407

In [33]:
ex_data.DR()

0.0196078431372549

In [34]:
ex_data.DE()

0.02

In [35]:
ex_data.growth_ni()

-8.70738830143214e-05

In [36]:
ex_data.FCFE()

1376.6223529411768

In [37]:
ex_data.FCFF()

350.43399999999997

In [38]:
ex_data.net_capex()

67.97000000000003

In [39]:
ex_data.pnl.loc['Depreciation And Amortisation Expenses','Mar 21']

'433.65'

In [42]:
ex_data.EBIT()

1217.22

In [43]:
ex_data.change_wc()

-652.3100000000004

In [44]:
ex_data.r()

0.1369196335510024

In [45]:
ex_data.r_equity()

0.13878508470175407

In [48]:
g = ex_data.reinvestment()*ex_data.roc()

In [47]:
ex_data.rev_growth()

0.17704037980699794

In [52]:
#assuming that exide will do well with the arrival of EVs and r will become lower (12%) for stable growth
five_year_gs = np.linspace(g,g+0.06,5)

In [62]:
numerator = (five_year_gs+1).cumprod()

In [58]:
r_ex = ex_data.r()

In [60]:
denominator = np.array([(1+r_ex)**i for i in range(1,6)])

In [61]:
denominator

array([1.13691963, 1.29258625, 1.46956669, 1.67077922, 1.8995417 ])

In [66]:
year_factors = numerator/denominator

In [67]:
year_factors

array([0.91281362, 0.84527196, 0.79388003, 0.75608678, 0.73006817])

In [68]:
ex_data.FCFF()

350.43399999999997

In [71]:
terminal_val = ex_data.FCFF()*numerator[4]*(1+0.05)/(0.12 - 0.05)

In [72]:
ex_data.FCFF()*(year_factors.sum()) 

1415.0947427136157

In [73]:
terminal_val

7289.701469694377

In [75]:
total_worth = ex_data.FCFF()*(year_factors.sum()) + terminal_val

In [77]:
share_price = (10**7)*total_worth/ex_data.shares

In [78]:
print(f'{ex_data.comp} has a price per share of ₹{round(share_price,2)}')

Exide Industries Limited has a price per share of ₹102.41


## HDFC Bank

In [61]:
hdfc_kundali = stock_ki_kundali('HDFC Bank',['ratios','profit and loss statement','balance sheet'])

h_df1 = hdfc_kundali[0]
h_df2 = hdfc_kundali[1]
h_df3 = hdfc_kundali[2]

In [64]:
h_df2

Profit & Loss account of HDFC Bank (in Rs. Cr.),Mar 21,Mar 20,Mar 19,Mar 18,Mar 17
,12 mths,12 mths,12 mths,12 mths,12 mths
INCOME,,,,,
Interest / Discount on Advances / Bills,"102,299.13","99,079.63","83,736.16","67,658.90","55,986.18"
Income from Investments,"23,211.62","20,572.73","19,924.75","16,229.79","15,951.56"
Interest on Balance with RBI and Other Inter-Bank funds,"2,414.30","1,868.77",660.62,540.62,544.86
Others,627.35,668.17,839.21,858.53,788.76
Total Interest Earned,"128,552.40","122,189.29","105,160.74","85,287.84","73,271.35"
Other Income,"27,332.88","24,878.97","18,947.05","16,056.60","12,877.63"
Total Income,"155,885.28","147,068.27","124,107.79","101,344.45","86,148.99"
EXPENDITURE,,,,,


In [81]:
hdfc_data = valuer(betas2021,companies2021,erpdata2021,'HDFC Bank',h_df1,h_df2,h_df3,capex=1617.38,dcf_type='equity',shares=5512776482)

In [82]:
hdfc_data.industry_beta_unlevered()

2.034328529022314

In [89]:
profits = (h_df2.loc['Net Profit / Loss for The Year',:].apply(lambda s:float(s.replace(',',''))))

In [93]:
ni_gr = (profits[0]/profits[-1])**0.25 - 1

In [97]:
dis = hdfc_data.erp()*hdfc_data.industry_beta_unlevered() + hdfc_data.risk_free_rate

In [105]:
gs=np.linspace(ni_gr,ni_gr-0.08,5)

In [107]:
facs = (gs+1).cumprod()/np.array([(1+dis)**i for i in range(1,6)])

In [112]:
eps0 = h_df1.loc['Diluted EPS (Rs.)',:].astype(float)[0]

In [118]:
#Assume r = 0.15 and g = 6% in the long term
eps0*facs.sum() + eps0*facs[-1]*1.06/(0.1-0.06)

1677.2551937753879

## Asian Paints

In [53]:
ap_kundali = stock_ki_kundali('Asian Paints',['ratios','profit and loss statement','balance sheet'])

ap_df1 = ap_kundali[0]
ap_df2 = ap_kundali[1]
ap_df3 = ap_kundali[2]

In [54]:
ap_df1

Key Financial Ratios of Asian Paints (in Rs. Cr.),Mar 21,Mar 20,Mar 19,Mar 18,Mar 17
Per Share Ratios,,,,,
Basic EPS (Rs.),32.73,28.20,22.48,21.26,20.22
Diluted EPS (Rs.),32.73,28.20,22.48,21.26,20.22
Cash EPS (Rs.),41.38,36.53,29.08,25.15,23.99
Book Value [ExclRevalReserve]/Share (Rs.),137.92,109.82,102.50,91.10,83.19
Book Value [InclRevalReserve]/Share (Rs.),137.92,109.82,102.50,91.10,83.19
Revenue from Operations/Share (Rs.),226.36,210.71,200.59,175.40,157.03
PBDIT/Share (Rs.),53.78,46.56,41.69,35.64,33.87
PBIT/Share (Rs.),45.53,38.42,35.20,31.88,30.38
PBT/Share (Rs.),44.58,37.36,34.10,31.51,30.07


In [94]:
ap_data = valuer(betas2021,companies2021,erpdata2021,'Asian Paints',ap_df1,ap_df2,ap_df3,capex=288.71,dcf_type='firm',shares=95919779)

In [95]:
ap_data.r()

0.12097286086539372

In [59]:
ap_data.r_equity()

0.12329263441095298

In [61]:
ap_data.FCFE()

849.55776699029

In [62]:
ap_data.FCFF()

2768.4559999999997

In [64]:
ap_data.growth_ni()

0.12748917438575558

In [66]:
ap_data.rev_growth()

0.23990398479535235

In [68]:
ap_data.beta()

1.1732903791472213

In [69]:
ap_data.DE()

0.03

In [70]:
ap_data.DR()

0.029126213592233007

In [71]:
ap_data.balance

Balance Sheet of Asian Paints (in Rs. Cr.),Mar 21,Mar 20,Mar 19,Mar 18,Mar 17
Total Share Capital,95.92,95.92,95.92,95.92,95.92
Total Reserves and Surplus,12710.37,10034.24,9374.63,8314.31,7507.97
Total Shareholders Funds,12806.29,10130.16,9470.55,8410.23,7603.89
Total Non-Current Liabilities,1214.61,1240.70,1266.20,594.13,557.14
Total Current Liabilities,5925.86,4380.38,5180.10,4451.07,3884.87
Total Capital And Liabilities,20369.62,16154.77,16278.10,13783.08,12421.35
Fixed Assets,5738.87,6092.56,6384.93,4810.09,3367.80
Total Non-Current Assets,8343.02,8574.66,8651.15,6869.00,5455.67
Total Current Assets,12026.60,7580.11,7626.95,6914.08,6965.68
Total Assets,20369.62,16154.77,16278.10,13783.08,12421.35


In [73]:
ap_data.ratios

Key Financial Ratios of Asian Paints (in Rs. Cr.),Mar 21,Mar 20,Mar 19,Mar 18,Mar 17
Per Share Ratios,,,,,
Basic EPS (Rs.),32.73,28.20,22.48,21.26,20.22
Diluted EPS (Rs.),32.73,28.20,22.48,21.26,20.22
Cash EPS (Rs.),41.38,36.53,29.08,25.15,23.99
Book Value [ExclRevalReserve]/Share (Rs.),137.92,109.82,102.50,91.10,83.19
Book Value [InclRevalReserve]/Share (Rs.),137.92,109.82,102.50,91.10,83.19
Revenue from Operations/Share (Rs.),226.36,210.71,200.59,175.40,157.03
PBDIT/Share (Rs.),53.78,46.56,41.69,35.64,33.87
PBIT/Share (Rs.),45.53,38.42,35.20,31.88,30.38
PBT/Share (Rs.),44.58,37.36,34.10,31.51,30.07


In [75]:
ap_data.roc()

0.31768

In [76]:
ap_data.reinvestment()

0.09443713556934756

In [77]:
ap_data.reinvestment()*ap_data.roc()

0.030000789227670334

In [79]:
ap_data.rev_growth()

0.23990398479535235

In [81]:
g0 = ap_data.growth_ni()

In [86]:
r_ap = ap_data.r_equity()

In [87]:
r_ap

0.12329263441095298

In [88]:
terminal_val = ap_data.FCFE()*1.06*(g0/r_ap)**5/(r_ap - 0.06)

In [89]:
terminal_val

16820.01927599389

In [90]:
dcfs = ap_data.FCFE()*(g0/r_ap)*(1-(g0/r_ap)**5)/(1-(g0/r_ap))

In [91]:
dcfs

4701.731964038878

In [96]:
(terminal_val+dcfs)*(10**7)/ap_data.shares

2243.724022762059

## HDFC

In [20]:
hdfc = stock_ki_kundali('HDFC Bank',['ratios','profit and loss statement','balance sheet'],con=False)

In [21]:
hdfc_ratios = hdfc[0]
hdfc_pnl = hdfc[1]
hdfc_bal = hdfc[2]

In [31]:
hdfc_ratios.loc['Dividend/Share (Rs.)'].astype(float)/hdfc_ratios.loc['Basic EPS (Rs.)'].astype(float)

Key Financial Ratios of HDFC Bank (in Rs. Cr.)
Mar 21    0.000000
Mar 20    0.052072
Mar 19    0.190718
Mar 18    0.191854
Mar 17    0.192375
dtype: float64

In [32]:
payout = 19.15/100

In [33]:
roe = 15.5/100

In [39]:
hdfc_ratios.loc['Basic EPS (Rs.)',:].astype(float)

Key Financial Ratios of HDFC Bank (in Rs. Cr.)
Mar 21    56.60
Mar 20    48.01
Mar 19    78.65
Mar 18    67.76
Mar 17    57.18
Name: Basic EPS (Rs.), dtype: float64

In [40]:
hdfc_ratios.loc['Basic EPS (Rs.)',:].astype(float).mean()

61.64

In [41]:
hdfc_ratios.loc['Basic EPS (Rs.)',:].astype(float).median()

57.18

In [42]:
hdfc_ratios.loc['Basic EPS (Rs.)',:].astype(float).std()

11.80957873931158

In [44]:
eps0 = hdfc_ratios.loc['Basic EPS (Rs.)',:].astype(float)['Mar 21']

In [45]:
eps0

56.6

In [46]:
g = roe*(1-payout)

In [47]:
g

0.1253175

In [51]:
erpdata2021.loc['India']

GDP (in billions) in 2019      2875.1
Moody's rating                   Baa3
Sovereign CDS                  0.0124
Adj. Default Spread          0.019453
Equity Risk Premium          0.068519
Country Risk Premium         0.021319
Corporate Tax Rate                0.3
Region                           Asia
Name: India, dtype: object

In [54]:
companies2021

,Company Name,Exchange:Ticker,Industry Group,Country,Broad Group,Sub Group
0,Activation Group Holdings Limited (SEHK:9919),SEHK:9919,Advertising,China,Emerging Markets,China
1,Adamant DRI Processing and Minerals Group (OTC...,OTCPK:ADMG,Advertising,China,Emerging Markets,China
2,Adtiger Corporations Limited (SEHK:1163),SEHK:1163,Advertising,China,Emerging Markets,China
3,Affle (India) Limited (NSEI:AFFLE),NSEI:AFFLE,Advertising,India,Emerging Markets,India
4,Afromedia Plc (NGSE:AFROMEDIA),NGSE:AFROMEDIA,Advertising,Nigeria,Emerging Markets,Africa and Middle East
...,...,...,...,...,...,...
23675,Luduson G Inc. (OTCPK:LDSN),OTCPK:LDSN,NaN,Hong Kong,Emerging Markets,China
23676,PP Miletic AD (BELEX:PMLT),BELEX:PMLT,NaN,Serbia,Emerging Markets,Eastern Europe & Russia
23677,SOS Limited (NYSE:SOS),NYSE:SOS,NaN,China,Emerging Markets,China
23678,"Tianjin Faw Xiali Automobile Co.,Ltd. (SZSE:00...",SZSE:000927,NaN,China,Emerging Markets,China


In [55]:
companies2021[companies2021['Company Name'].str.match('(^HDFC.*)')==True]

,Company Name,Exchange:Ticker,Industry Group,Country,Broad Group,Sub Group
2038,HDFC Bank Limited (NSEI:HDFCBANK),NSEI:HDFCBANK,Bank (Money Center),India,Emerging Markets,India
14164,HDFC Life Insurance Company Limited (BSE:540777),BSE:540777,Insurance (Life),India,Emerging Markets,India
14513,HDFC Asset Management Company Limited (NSEI:HD...,NSEI:HDFCAMC,Investments & Asset Management,India,Emerging Markets,India


In [80]:
bank_beta = betas2021.loc['Bank (Money Center)','Average Levered Beta']

In [81]:
hdfc_beta = 1.29

In [82]:
betas2021.loc['Investments & Asset Management','Average Levered Beta']

0.6727983202814664

In [83]:
indian_risk_free = 0.04

In [84]:
erpdata2021.loc['United States','Equity Risk Premium']

0.0472

In [93]:
r_equity_hdfc = indian_risk_free + hdfc_beta*(erpdata2021.loc['United States','Equity Risk Premium'] + erpdata2021.loc['India','Country Risk Premium'])

In [94]:
r_equity_hdfc

0.1283894560402907

In [95]:
mature_beta = 1

In [96]:
#For first 5 years
ratio = (1+g)/(1+r_equity_hdfc)

In [97]:
#Next 5 years
mature_growth = 3.9/100

In [99]:
r_mature = indian_risk_free + mature_beta*(erpdata2021.loc['United States','Equity Risk Premium'] + erpdata2021.loc['India','Country Risk Premium'])

In [100]:
r_mature

0.10851895817076798

In [101]:
roe_final = 16/100

In [102]:
payout_final = 1 - (mature_growth/roe_final)

In [103]:
payout_final

0.75625

In [104]:
payout

0.19149999999999998

In [106]:
payouts = np.linspace(payout,payout_final,5)

In [107]:
growths = np.linspace(g,mature_growth,5)

In [108]:
growths

array([0.1253175 , 0.10373813, 0.08215875, 0.06057938, 0.039     ])

In [111]:
#growth stage
dcfs = eps0*payout*ratio*(1-ratio**5)/(1-ratio)

In [112]:
dcfs

53.75348195571481

In [113]:
eps0

56.6

In [114]:
ratio

0.9972775746672866

In [115]:
eps0

56.6

In [116]:
payout

0.19149999999999998

In [117]:
#steadying stage
eps5 = eps0*(1+g)**5

In [118]:
eps5

102.13904543432218

In [119]:
(1+growths).cumprod()

array([1.1253175 , 1.24205583, 1.34410158, 1.42552642, 1.48112195])

In [120]:
payouts

array([0.1915   , 0.3326875, 0.473875 , 0.6150625, 0.75625  ])

In [124]:
eps5*(1+growths).cumprod()*payouts

array([ 22.01079078,  42.20553357,  65.05604904,  89.55427312,
       114.40578866])

In [125]:
discount_factors = np.array([(1+r_equity_hdfc)**i for i in range(6,11)])

In [126]:
discount_factors

array([2.06421113, 2.32923408, 2.62828317, 2.96572702, 3.3464951 ])

In [129]:
dcfs_steady = eps5*(1+growths).cumprod()*payouts/discount_factors

In [131]:
dcfs_steady.sum() 

117.91841671542129

In [132]:
eps10 = eps5*(1+growths).cumprod()[-1]

In [136]:
terminal_val = eps10*(1+mature_growth)*payout_final/((r_mature - mature_growth)*(1+r_equity_hdfc)**10)

In [137]:
final_val = terminal_val + dcfs_steady.sum() + dcfs

In [138]:
final_val

682.6121277391898

## HDFC Asset Management Company

In [141]:
hdfc_amc = stock_ki_kundali('HDFC AMC',['ratios','profit and loss statement','balance sheet'],con=False)

In [145]:
amc_ratios=hdfc_amc[0]

In [149]:
amc_ratios.loc['Dividend / Share(Rs.)'].astype(float)

Key Financial Ratios of HDFC Asset Management Company (in Rs. Cr.)
Mar 20    28.0
Mar 19    24.0
Mar 18    16.0
Mar 17    92.0
Mar 16    80.0
Name: Dividend / Share(Rs.), dtype: float64

In [153]:
amc_ratios.loc['Basic EPS (Rs.)'].astype(float)

Key Financial Ratios of HDFC Asset Management Company (in Rs. Cr.)
Mar 20    59.37
Mar 19    43.87
Mar 18    34.52
Mar 17    27.33
Mar 16    23.64
Name: Basic EPS (Rs.), dtype: float64

In [154]:
amc_ratios.loc['Dividend / Share(Rs.)'].astype(float)/amc_ratios.loc['Basic EPS (Rs.)'].astype(float)

Key Financial Ratios of HDFC Asset Management Company (in Rs. Cr.)
Mar 20    0.471619
Mar 19    0.547071
Mar 18    0.463499
Mar 17    3.366264
Mar 16    3.384095
dtype: float64

In [157]:
amc_ratios.loc['Return on Networth / Equity (%)',:].astype(float).mean()

34.669999999999995

In [158]:
amc_ratios.loc['Return on Networth / Equity (%)',:].astype(float).median()

31.55

In [159]:
amc_ratios.loc['Return on Networth / Equity (%)',:].astype(float).std()

5.066867868812052

In [160]:
roe_amc = 34.7/100

In [187]:
amc_g = roe_amc*(1-0.48)

In [194]:
amc_eps0=amc_ratios.loc['Basic EPS (Rs.)',:].astype(float)['Mar 20']

In [195]:
amc_eps0

59.37

In [177]:
companies2021[companies2021['Company Name'].str.match('(^HDFC Asset.*)')==True]

,Company Name,Exchange:Ticker,Industry Group,Country,Broad Group,Sub Group
14513,HDFC Asset Management Company Limited (NSEI:HD...,NSEI:HDFCAMC,Investments & Asset Management,India,Emerging Markets,India


In [178]:
betas2021.loc['Investments & Asset Management',:]

0
Number of firms                              78
Average Unlevered Beta                 0.552355
Average Levered Beta                   0.672798
Average correlation with the market     0.18511
Total Unlevered Beta                    2.98393
Total Levered Beta                      3.63459
Name: Investments & Asset Management, dtype: object

In [179]:
amc_beta = betas2021.loc['Investments & Asset Management','Average Levered Beta']

In [180]:
r_equity_hdfc_amc = indian_risk_free + amc_beta*(erpdata2021.loc['United States','Equity Risk Premium'] + erpdata2021.loc['India','Country Risk Premium'])

In [181]:
r_equity_hdfc_amc

0.08609943996472876

In [182]:
roe_stable = 0.2

In [183]:
amc_payout_stable = 1 - 0.05/0.2

In [184]:
payout_stable

0.75

In [188]:
amc_payout = 0.48

In [190]:
amc_g

0.18044000000000002

In [191]:
g_amc_stable = 0.05

In [205]:
#Growth stage : 5 years
dividend_amc = amc_eps0*amc_payout

In [206]:
dividend_amc

28.4976

In [207]:
amc_ratio = (1+amc_g)/(1+r_equity_hdfc_amc)

In [208]:
amc_ratio

1.0868618070904583

In [209]:
dcf1 = dividend_amc*amc_ratio*(1-amc_ratio**5)/(1-amc_ratio)

In [210]:
dcf1

184.20858990358693

In [211]:
payouts_amc = np.linspace(amc_payout,payout_stable,5)

In [212]:
payouts_amc

array([0.48  , 0.5475, 0.615 , 0.6825, 0.75  ])

In [214]:
growth_amc = np.linspace(amc_g,g_amc_stable,5)

In [215]:
growth_amc

array([0.18044, 0.14783, 0.11522, 0.08261, 0.05   ])

In [216]:
amc_eps5 = amc_eps0*(1+amc_g)**5

In [224]:
amc_eps5*(1+growth_amc).cumprod()*payouts_amc

array([ 77.10309136, 100.94672842, 126.45726655, 151.92993929,
       175.30377611])

In [225]:
discount_factors = np.array([1/(1+r_equity_hdfc_amc)**i for i in range(6,11)])

In [226]:
discount_factors

array([0.60923166, 0.56093543, 0.51646784, 0.47552537, 0.43782857])

In [230]:
dcf2 = (amc_eps5*(1+growth_amc).cumprod()*payouts_amc*discount_factors).sum()

In [231]:
dcf2

317.9088946551354

In [232]:
eps10 = amc_eps5*(1+growth_amc).cumprod()[-1]

In [235]:
final_dividend = eps10*0.75

In [236]:
final_dividend

175.3037761062171

In [237]:
terminal_val = final_dividend*(1.05)/((r_equity_hdfc_amc - 0.05)*(1+r_equity_hdfc_amc)**10)

In [238]:
terminal_val+dcf2+dcf1

2734.580167501751

## ICICI Securities

In [11]:
isec = stock_ki_kundali('ICICI Securities',['ratios','profit and loss statement','balance sheet'],con=False)

In [14]:
isec_ratios = isec[0]

In [16]:
isec_ratios.loc['Dividend / Share(Rs.)'].astype(float)

Key Financial Ratios of ICICI Securities Ltd. (in Rs. Cr.)
Mar 20    11.00
Mar 19     9.40
Mar 18     3.90
Mar 17     2.55
Mar 16     2.00
Name: Dividend / Share(Rs.), dtype: object

In [19]:
isec_ratios.loc['Dividend / Share(Rs.)'].astype(float)/isec_ratios.loc['Basic EPS (Rs.)'].astype(float)

Key Financial Ratios of ICICI Securities Ltd. (in Rs. Cr.)
Mar 20    0.660264
Mar 19    0.621693
Mar 18    0.228739
Mar 17    0.243321
Mar 16    0.273224
dtype: float64

In [20]:
isec_payout = 60/100

In [23]:
isec_ratios.loc['Return on Networth / Equity (%)'].astype(float)

Key Financial Ratios of ICICI Securities Ltd. (in Rs. Cr.)
Mar 20    45.04
Mar 19    47.07
Mar 18    65.47
Mar 17    69.60
Mar 16    59.79
Name: Return on Networth / Equity (%), dtype: float64

In [24]:
isec_ratios.loc['Return on Networth / Equity (%)'].astype(float).mean()

57.39399999999999

In [25]:
isec_ratios.loc['Return on Networth / Equity (%)'].astype(float).std()

10.944808358303948

In [26]:
isec_ratios.loc['Return on Networth / Equity (%)'].astype(float).median()

59.79

In [27]:
roe0 = 0.5

In [28]:
g = roe0*(1-isec_payout)

In [29]:
g

0.2

In [32]:
ind_risk_free = 5.1/100

In [33]:
stable_g = 0.05

In [36]:
companies2021[companies2021['Company Name'].str.match('(^ICICI.*)')==True]

,Company Name,Exchange:Ticker,Industry Group,Country,Broad Group,Sub Group
2048,ICICI Bank Limited (NSEI:ICICIBANK),NSEI:ICICIBANK,Bank (Money Center),India,Emerging Markets,India
2804,ICICI Securities Limited (NSEI:ISEC),NSEI:ISEC,Brokerage & Investment Banking,India,Emerging Markets,India
14166,ICICI Prudential Life Insurance Company Limite...,BSE:540133,Insurance (Life),India,Emerging Markets,India
14270,ICICI Lombard General Insurance Company Limite...,BSE:540716,Insurance (Prop/Cas.),India,Emerging Markets,India


In [37]:
betas2021.loc['Brokerage & Investment Banking',:]

0
Number of firms                             148
Average Unlevered Beta                 0.421148
Average Levered Beta                   0.539985
Average correlation with the market    0.147072
Total Unlevered Beta                    2.86354
Total Levered Beta                      3.67156
Name: Brokerage & Investment Banking, dtype: object

In [38]:
isec_beta = betas2021.loc['Brokerage & Investment Banking','Average Levered Beta']

In [40]:
r_eq = ind_risk_free + isec_beta*(erpdata2021.loc['United States','Equity Risk Premium'] + erpdata2021.loc['India','Country Risk Premium'])

In [41]:
r_eq

0.08799919964841585

In [52]:
eps0 = float(isec_ratios.loc['Basic EPS (Rs.)','Mar 20'])

In [53]:
eps0

16.66

In [58]:
np.linspace(g,stable_g,7)

array([0.2  , 0.175, 0.15 , 0.125, 0.1  , 0.075, 0.05 ])

In [69]:
growth_arr = np.hstack((np.array([0.2,0.2,0.2]),np.linspace(g,stable_g,7)))

In [70]:
isec_payout

0.6

In [71]:
final_payout = (1 - stable_g/0.2)

In [72]:
final_payout

0.75

In [73]:
payout_arr = np.hstack((np.array([0.6,0.6,0.6]),np.linspace(isec_payout,final_payout,7)))

In [74]:
payout_arr

array([0.6  , 0.6  , 0.6  , 0.6  , 0.625, 0.65 , 0.675, 0.7  , 0.725,
       0.75 ])

In [77]:
isec_df = pd.DataFrame({'Growth':growth_arr,'Payout':payout_arr,'Year':np.array(range(1,11)),'Discount factor':np.array([(1+r_eq)**-i for i in range(1,11)])})

In [78]:
isec_df

,Growth,Payout,Year,Discount factor
0,0.200,0.600,1,0.919118
1,0.200,0.600,2,0.844778
2,0.200,0.600,3,0.776451
3,0.200,0.600,4,0.713651
4,0.175,0.625,5,0.655929
5,0.150,0.650,6,0.602877
6,0.125,0.675,7,0.554115
7,0.100,0.700,8,0.509297
8,0.075,0.725,9,0.468105
9,0.050,0.750,10,0.430243


In [79]:
isec_df.set_index('Year',inplace=True)

In [82]:
isec_df['Compound factor'] = (1+isec_df['Growth']).cumprod()

In [83]:
isec_df

,Growth,Payout,Discount factor,Compound factor
Year,,,,
1,0.200,0.600,0.919118,1.200000
2,0.200,0.600,0.844778,1.440000
3,0.200,0.600,0.776451,1.728000
4,0.200,0.600,0.713651,2.073600
5,0.175,0.625,0.655929,2.436480
6,0.150,0.650,0.602877,2.801952
7,0.125,0.675,0.554115,3.152196
8,0.100,0.700,0.509297,3.467416
9,0.075,0.725,0.468105,3.727472


In [86]:
isec_df['Discounted dividends'] = isec_df['Discount factor']*isec_df['Compound factor']*isec_df['Payout']*eps0

In [87]:
isec_df

,Growth,Payout,Discount factor,Compound factor,Discounted dividends
Year,,,,,
1,0.200,0.600,0.919118,1.200000,11.025008
2,0.200,0.600,0.844778,1.440000,12.159944
3,0.200,0.600,0.776451,1.728000,13.411713
4,0.200,0.600,0.713651,2.073600,14.792342
5,0.175,0.625,0.655929,2.436480,16.640830
6,0.150,0.650,0.602877,2.801952,18.292691
7,0.125,0.675,0.554115,3.152196,19.642282
8,0.100,0.700,0.509297,3.467416,20.594456
9,0.075,0.725,0.468105,3.727472,21.075126


In [99]:
terminal_val = isec_df.loc[10,'Compound factor']*eps0*0.75*(1+stable_g)*isec_df.loc[10,'Discount factor']/(r_eq-stable_g)

In [102]:
terminal_val + isec_df['Discounted dividends'].sum()

750.0667499375982